In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

In [10]:
def hamConverter(value):
    return 0 if value == "ham" else 1

In [11]:
def get_best_score(vectorizer, ngrams=(1, 1), verbose=True):
    vectorizer = vectorizer(ngram_range=ngrams)
    X = vectorizer.fit_transform(data.msg)
    
    best_res = -1
    best_c = -1

    for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
        cls = LogisticRegression(C=c)
        res = cross_val_score(cls, X, data.is_spam, scoring="f1_macro", cv=10)
        if verbose:
            print c, '\t', np.mean(res), np.std(res)

        if np.mean(res) > best_res:
            best_c = c
            best_res = np.mean(res)
            
    return best_res, best_c, X

In [12]:
data = pd.read_csv("SMSSpamCollection.txt", sep='\t', header = None, names = ["is_spam", "msg"], converters={"is_spam": hamConverter})

In [13]:
data.head()

,is_spam,msg
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
ans1, c1, X = get_best_score(CountVectorizer)

0.01 	0.878490605033 0.0278790215221
0.1 	0.945341388869 0.0127270570793
0.5 	0.958048828369 0.0114923541911
1 	0.961439883123 0.0111136136931
5 	0.965630631432 0.0100254597851
10 	0.964768692811 0.0105745573823
100 	0.965519045969 0.0104983807255
200 	0.96512703282 0.0106615483688
500 	0.965519154006 0.0104981363784
1000 	0.965900915279 0.0105019673784
10000 	0.965515354099 0.0108273920481
15000 	0.965916069386 0.0114287871959
20000 	0.965916069386 0.0114287871959
100000 	0.967613684947 0.0112335616015


In [15]:
print "Answer 1: "
print ans1

Answer 1: 
0.967613684947


In [19]:
cls = LogisticRegression(C=c1).fit(X, data.is_spam)

samples = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time",
            "Have you visited the last lecture on physics?",
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
            "Only 99$"]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data.msg)
sample = vectorizer.transform(samples)
res = cls.predict(sample)
print "Answer 2: "
print " ".join([str(i) for i in res])
# print cls.predict_proba(sample)

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [ ]:
scores1 = []

scores1.append(get_best_score(CountVectorizer, (2, 2), False))
scores1.append(get_best_score(CountVectorizer, (3, 3), False))
scores1.append(get_best_score(CountVectorizer, (1, 3), False))

In [ ]:
print "Answer 3: "
print " ".join(["{0:0.2f}".format(i) for i, _ in scores1])

In [ ]:
scores2 = []
cls = MultinomialNB().fit(X, data.is_spam)


for i in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=i)
    X = vectorizer.fit_transform(data.msg)
    
    res = cross_val_score(cls, X, data.is_spam, scoring="f1_macro", cv=10)
    
    scores2.append(np.mean(res))

In [ ]:
print "Answer 4: "
print " ".join(["{0:0.2f}".format(i) for i in scores2])

In [ ]:
ans2, _ = get_best_score(TfidfVectorizer)

In [ ]:
print "Answer 5: "
print ans2